In [1]:
import pandas as pd
import json
import os

# Directory containing your Excel files
directory_path = 'raw_questions'  # Update this to your directory's path

# List to hold all questions and their contexts
all_questions = []

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx'):  # Check if the file is an Excel file
        file_path = os.path.join(directory_path, filename)
        df = pd.read_excel(file_path)
        
        # Select the required columns
        required_columns = ['Question', 'Irrelevant Context', 'Vague Context', 'Relevant Context']
        if set(required_columns).issubset(df.columns):
            df_selected = df[required_columns]
            questions_json = df_selected.to_dict(orient='records')
            all_questions.extend(questions_json)

# Convert to JSON string for pretty printing or saving to a file
json_str = json.dumps(all_questions, indent=4)

# Save to a JSON file
output_file_path = 'combined_questions_bank.json'  # You can change the file name as needed
with open(output_file_path, 'w') as f:
    f.write(json_str)

print("Combined JSON file has been created successfully.")


Combined JSON file has been created successfully.


In [1]:
import json
import re

# Load the content of both JSON files
with open("Adjusted_Generated_answer_bank_with_context_ids.json", "r") as file:
    adjusted_generated = json.load(file)

with open("combined_questions_bank.json", "r") as file:
    combined_questions = json.load(file)

# Create a new list to hold the updated elements
updated_elements = []

# Loop through each element in the adjusted_generated list
for adj_element in adjusted_generated:
    question_to_find = adj_element["question"]
    context_type_to_find = adj_element["context_type"]

    # Find the matching question and context type in the combined_questions list
    found=False
    for comb_element in combined_questions:
        question = comb_element["Question"].lower()  # Convert to lowercase first

        # Remove all whitespaces
        question_no_spaces = re.sub(r"\s+", "", question)
        # Remove heading for one question
        question_no_heading = re.sub(r"#17:\s*graduate\s*quantum", "", question_no_spaces)
        # Remove "problem:" if it appears at the beginning
        final_question = re.sub(r"^problem:", "", question_no_heading)
        if final_question == re.sub(r"\s+", "", question_to_find.lower()):
            # Determine the matching context to add based on the context type
            if context_type_to_find == "irrelevant":
                context_to_add = comb_element["Irrelevant Context"]
            elif context_type_to_find == "vague":
                context_to_add = comb_element["Vague Context"]
            elif context_type_to_find == "relevant":  
                context_to_add = comb_element["Relevant Context"]
            else:
                context_to_add = ""
            
            # Create a new element with added context and append it to the list
            new_element = adj_element.copy()
            new_element["context"] = context_to_add
            updated_elements.append(new_element)
            found=True
            break  # Exit the loop after finding and handling the matching question
    if not found:
        print(question_to_find)

# Save the new list to a JSON file
new_json_path = "final_question_context_response.json"
with open(new_json_path, "w") as file:
    json.dump(updated_elements, file, indent=4)


In [5]:
# Load the JSON data
with open("corrected_updated_questionjson.json", "r") as file:
    data = json.load(file)

# Find and print the IDs of elements with any empty fields
ids_with_empty_fields = [element['id'] for element in data if not all(element.values())]

print("IDs of elements with empty fields:")
for id in ids_with_empty_fields:
    print(id)

IDs of elements with empty fields:
